In [2]:
import cv2
import numpy as np
import apriltag 
import matplotlib.pyplot as plt
import os
import datetime

# AprilTag detector Options
options = apriltag.DetectorOptions(families='tag36h11')

# AprilTag detector Options
detector = apriltag.Detector(options)

#Set Directory
image_path = '/Users/kanjisugimori/Documents/VS_Code/img.jpg'

# Directory to save images with AprilTag detections
output_directory = "output_images"

# Create the output directory if it doesn't exist
if not os.path.exists(image_path):
    os.makedirs(image_path)

# Initialize the camera (you may need to change the camera index)
cap = cv2.VideoCapture(0)

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()

    if ret:
        # Convert the frame to grayscale for AprilTag detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect AprilTags in the frame
        tags = detector.detect(gray)

        if len(tags) > 0:
            # Capture the current date and time for the image filename
            current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

            # Save the image with the detected AprilTag to the specified directory
            image_filename = os.path.join(output_directory, f"apriltag_{current_time}.jpg")
            cv2.imwrite(image_filename, frame)

            print(f"AprilTag detected. Image saved as {image_filename}")

    # Display the video feed (optional)
    cv2.imshow("AprilTag Detection", frame)

    # Press 'q' to exit the program
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the OpenCV windows
cap.release()
cv2.destroyAllWindows()

# Function to draw a cube given the tag's pose
def draw_cube(image, rvec, tvec, K):

    cube_points_3d = np.array([
        [0.5, 0.5, 0],
        [-0.5, 0.5, 0],
        [-0.5, -0.5, 0],
        [0.5, -0.5, 0],
        [0.5, 0.5, -1.0],
        [-0.5, 0.5, -1.0],
        [-0.5, -0.5, -1.0],
        [0.5, -0.5, -1.0]
    ], dtype=np.float32)
    # Project cube vertices onto the image plane
    cube_points_2d, _ = cv2.projectPoints(cube_points_3d, rvec, tvec, K, None)

    # Draw cube edges
    cube_points_2d = np.int32(cube_points_2d).reshape(-1, 2)
    for i in range(4):
        j = (i + 1) % 4
        cv2.line(image, tuple(cube_points_2d[i]), tuple(cube_points_2d[j]), (0, 0, 255), 2)
        cv2.line(image, tuple(cube_points_2d[i + 4]), tuple(cube_points_2d[j + 4]), (0, 0, 255), 2)
        cv2.line(image, tuplPoe(cube_points_2d[i]), tuple(cube_points_2d[i + 4]), (0, 0, 255), 2)
    for i in range(4):
        cv2.line(image, tuple(cube_points_2d[i]), tuple(cube_points_2d[i + 4]), (0, 0, 255), 2)

    return image

# Load an image (replace 'your_image.jpg' with the actual image file)
image = cv2.imread(image_path)

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#cv2.imshow("image",gray)
#cv2.waitKey(0)

# Detect AprilTags in the image
detections = detector.detect(gray)

#print("[INFO] {} total AprilTags detected".format(len(detections)))
#print(detections)
# Check if any AprilTags were detected
if detections:
    # Assuming you have only one tag in the image, you can access its pose
    tag = detections[0]

    # Camera intrinsic matrix (replace with your actual camera matrix)
    K = np.array([[1.49082247e+03, 0, 9.89406025e+02],
              [0, 1.49799612e+03, 5.71826510e+02],
              [0, 0, 1]])
    print(K)
    # Distortion coefficients (replace with your actual distortion coefficients)
    dist_coeffs = np.array([-1.96472816e+00, 9.61621868e+01, -1.80150196e-02, 1.36373363e-02, -1.34299646e+03])
    print(tag.corners)
    # Use cv2.solvePnP to estimate the pose (rvec and tvec)
    corner=np.array(tag.corners)
    # Define the 3D coordinates of the cube's vertices
    # Assuming the AprilTag is a square with unit side length
    points_3d = np.array([
    [0.5, 0.5, 0],
    [-0.5, 0.5, 0],
    [-0.5, -0.5, 0],
    [0.5, -0.5, 0], ],dtype = np.float64)

    success,rvec,tvec = cv2.solvePnP(points_3d,corner,K,dist_coeffs)

    if success:
        # Print the rotation vector (rvec) and translation vector (tvec)
        print("Rotation Vector (rvec):")
        print(rvec)
        print("Translation Vector (tvec):")
        print(tvec)

        # Draw the cube on the image
        image_with_cube = draw_cube(image.copy(), rvec, tvec, K)
        
        # Display the image with the cube
        cv2.imshow('Image with Cube', image_with_cube)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("cv2.solvePnP failed to estimate the pose.")

else:
    print("No AprilTags detected in the image.")


[[1.49082247e+03 0.00000000e+00 9.89406025e+02]
 [0.00000000e+00 1.49799612e+03 5.71826510e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[ 770.269104    647.71081543]
 [ 889.64642334  387.80889893]
 [1138.50842285  469.32467651]
 [1009.80987549  757.58508301]]
Rotation Vector (rvec):
[[0.20931102]
 [0.5522823 ]
 [1.92159658]]
Translation Vector (tvec):
[[-0.14361898]
 [-0.02957997]
 [ 4.89719302]]


NameError: name 'tuplPoe' is not defined

: 